# Common Python Data Structures (Guide)

[https://realpython.com/python-data-structures/](https://realpython.com/python-data-structures/)

Neste tutorial, você aprenderá:

- Quais tipos de dados abstratos comuns são integrados à biblioteca padrão do Python
- Como os tipos de dados abstratos mais comuns são mapeados para o esquema de nomenclatura do Python
- Como colocar tipos de dados abstratos em uso prático em vários algoritmos

## Registros, Estuturas e Objetos de Transferência de Dados

Em comparação com os arrays, as estruturas de dados de registro (*record*) fornecem um número fixo de campos. Cada campo pode ter um nome e também pode ter um tipo diferente.

Nesta seção, você verá como implementar registros, estruturas(*struct*) e objetos de dados comuns no Python, usando apenas tipos de dados built-in e classes da biblioteca padrão.


> Nota: Aqui, vamos usar uma definição vaga de registro. Por exemplo, também
> vamos discutir tipos built-in como a tupla, que pode ou não ser considerada
> registros em um sentido estrito, já que não fornecem campos nomeados.

O Python oferece vários tipos de dados que você pode usar para implementar registros, estruturas e objetos de transferência de dados (*data transfer objects*). Nesta seção, você dará uma rápida olhada em cada implementação e suas características únicas. No final, você encontrará um resumo e um guia de decisão que ajudará você a fazer suas próprias escolhas.

### dict: Simple Data Objects

Como mencionado anteriormente, os dicionários do Python armazenam um número arbitrário de objetos, cada um identificado por uma chave exclusiva. Os dicionários também são frequentemente chamados de mapas ou matrizes associativas e permitem a pesquisa, a inserção e a exclusão eficientes de qualquer objeto associado a uma determinada chave.

Em Python, é possível usar dicionários como registro ou objeto de dados. Os dicionários são fáceis de criar, pois eles têm seu próprio açúcar sintático embutido na linguagem sob a forma de literais de dicionário. A sintaxe do dicionário é concisa e bastante conveniente para digitar.

Os objetos de dados criados usando dicionários são mutáveis, e há pouca proteção contra nomes de campo com erros de digitação, assim como campos podem ser adicionados e removidos livremente a qualquer momento. Ambas as propriedades podem introduzir bugs surpreendentes, e há sempre uma trade-off a ser feita entre conveniência e resiliência a erro.

In [1]:
car1 = {
    "color": "red",
    "mileage": 3812.4,
    "automatic": True,
}

car2 = {
    "color": "blue",
    "mileage": 40231,
    "automatic": False,
}

In [4]:
# car2.__repr__()
car2

{'color': 'blue', 'mileage': 40231, 'automatic': False}

In [3]:
# Get mileage
car2["mileage"]

40231

In [5]:
# Dicts são mutáveis
car2["mileage"] = 12
car2["windshield"] = "broken"
car2

{'color': 'blue', 'mileage': 12, 'automatic': False, 'windshield': 'broken'}

In [6]:
# Nenhuma proteção contra nomes de campo errados, campos ausentes ou
# campos extras
car3 = {
    "colr": "green",
    "automatic": False,
    "windshield": "broken",
}
car3

{'colr': 'green', 'automatic': False, 'windshield': 'broken'}

### tuple: Grupo de Objetos Imutáveis

As tuplas do Python são uma estrutura de dados simples para agrupar objetos arbitrários. Tuplas são imutáveis - elas não podem ser modificadas depois de criadas.

Sob o ponto de vista da performance, tuplas ocupam um pouco menos memória do que as listas em CPython, e eles também são mais rápidos para construir.

Como você pode ver na desmontagem abaixo, construir uma tupla leva a um único opcode LOAD_CONST, enquanto a construção de um objeto de lista com o mesmo conteúdo requer várias mais operações.

In [7]:
import dis  # noqa E402

dis.dis(compile("(23, 'a', 'b', 'c')", "", "eval"))

  1           0 LOAD_CONST               0 ((23, 'a', 'b', 'c'))
              2 RETURN_VALUE


In [8]:
dis.dis(compile("[23, 'a', 'b', 'c']", "", "eval"))

  1           0 BUILD_LIST               0
              2 LOAD_CONST               0 ((23, 'a', 'b', 'c'))
              4 LIST_EXTEND              1
              6 RETURN_VALUE


No entanto, você não deve colocar muita ênfase nessas diferenças. Na prática, a diferença de desempenho, na maioria das vezes, será insignificante. Tentar  espremer desempenho extra, de um programa, alternando as listas para tuplas, provavelmente será uma abordagem equivocada.

Uma possível desvantagem de tuplas simples é que os dados que você armazenam neles só podem ser retirados, acessando-o através de índices inteiros. Você não pode dar nomes para propriedades individuais armazenadas em uma tupla. Isso pode impactar a legibilidade de código.

Além disso, uma tupla é sempre uma estrutura ad-hoc: é difícil garantir que duas tuplas tenham o mesmo número de campos e as mesmas propriedades armazenadas nelas.

Isso facilita a introdução de bugs difíceis de perceber, como misturar a ordem de campo. Portanto, eu recomendaria que você mantenha o número de campos armazenados em uma tupla o mais baixo possível.

In [9]:
# Campos: color, mileage, automatic
car1 = ("red", 3812.4, True)
car2 = ("blue", 40231.0, False)

In [11]:
# car1.__repr__()
car1

('red', 3812.4, True)

In [12]:
# car2.__repr__()
car2

('blue', 40231.0, False)

In [13]:
import traceback  # noqa E402

try:
    # Tuplas são imutáveis
    car2[1] = 12
except TypeError:
    traceback.print_exc()

Traceback (most recent call last):
  File "C:\Users\josen\AppData\Local\Temp/ipykernel_38704/1258160115.py", line 5, in <module>
    car2[1] = 12
TypeError: 'tuple' object does not support item assignment


In [15]:
# Nenhuma proteção contra nomes de campo errados, campos ausentes ou
# campos extras
car3 = (3431.5, "green", True, "silver")
car3

(3431.5, 'green', True, 'silver')

### Escreva uma classe personalizada: mais trabalho, mais controle

Uma classe permite que você defina uma planta reutilizável para objetos de dados, para garantir que cada objeto forneça o mesmo conjunto de campos.

Usar classes regulares como tipos de dados registro é viável, mas também é necessário trabalho manual para obter as conveniências de outras implementações.Por exemplo, adicionar novos campos ao construtor `__init__` é verboso e leva tempo.

Além disso, a representação padrão de string para objetos instanciados a partir de classes personalizadas não é muito útil. Para corrigir isso, você pode ter que adicionar seu próprio método `__repr__` que, geralmente, é bastante detalhado e deve ser atualizado toda vez que você adicionar um novo campo.

Os campos das classes são mutáveis e novos campos podem ser adicionados livremente, quer você goste ou não. É possível ter mais controle de acesso e criar campos somente leitura usando o decorador `@property` mas, mais uma vez, isso requer escrever mais código de cola.

Escrever uma classe personalizada é uma ótima opção sempre que você quiser  adicionar lógica e comportamento de negócios aos seus objetos de registro usando métodos. No entanto, isso significa que esses objetos não são mais, tecnicamente, objetos de dados simples.

In [17]:
class Car:
    def __init__(self, color, mileage, automatic):
        self.color = color
        self.mileage = mileage
        self.automatic = automatic


car1 = Car("red", 3812.4, True)
car2 = Car("blue", 40231.0, False)

In [19]:
# Obtendo a propriedade mileage
car2.mileage

40231.0

In [23]:
# Classes são mutáveis
car2.mileage = 12
car2.windshield = "broken"

In [27]:
# A representação de string não é muito útil
# (o método __repr__ deve ser sobreescrito manualmente):
car2

Car(blue, 12, False)

In [28]:
def repr(self):
    return f"Car({self.color}, {self.mileage}, {self.automatic})"


Car.__repr__ = repr

car2

Car(blue, 12, False)